In [28]:
import os
import sys
import math
import joblib
import numpy as np
import pandas as pd
import json
import requests
from datetime import datetime, timedelta, time
from dateutil.relativedelta import relativedelta

In [52]:
sale = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', '2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx'), skiprows = 1)

In [53]:
sale['방송날'] = sale['방송일시'].dt.date
sale['방송월'] = sale['방송일시'].dt.month
sale['방송일'] = sale['방송일시'].dt.day
sale['방송시간'] = sale['방송일시'].dt.hour

In [54]:
sale['방송일시'] = sale.apply(lambda x : datetime.combine(x['방송날'], time(x['방송시간'])), axis = 1)

In [55]:
sale['방송일시']

0       2019-01-01 06:00:00
1       2019-01-01 06:00:00
2       2019-01-01 06:00:00
3       2019-01-01 06:00:00
4       2019-01-01 06:00:00
                ...        
38304   2020-01-01 00:00:00
38305   2020-01-01 00:00:00
38306   2020-01-01 01:00:00
38307   2020-01-01 01:00:00
38308   2020-01-01 01:00:00
Name: 방송일시, Length: 38309, dtype: datetime64[ns]

In [56]:
import math
sale['판매량'] = sale['취급액'] / sale['판매단가']
sale['판매량'] = sale['판매량'].fillna(0).apply(lambda x : math.ceil(x))

In [45]:
piv = sale[['방송일시', '판매량']].groupby('방송일시').mean().T

In [65]:
piv

방송일시,2019-01-01 06:00:00,2019-01-01 07:00:00,2019-01-01 08:00:00,2019-01-01 09:00:00,2019-01-01 10:00:00,2019-01-01 11:00:00,2019-01-01 12:00:00,2019-01-01 13:00:00,2019-01-01 14:00:00,2019-01-01 15:00:00,...,2019-12-31 16:00:00,2019-12-31 17:00:00,2019-12-31 18:00:00,2019-12-31 19:00:00,2019-12-31 20:00:00,2019-12-31 21:00:00,2019-12-31 22:00:00,2019-12-31 23:00:00,2020-01-01 00:00:00,2020-01-01 01:00:00
판매량,137.166667,270.666667,555.333333,384.333333,1083.666667,532.0,1052.333333,512.5,252.5,472.0,...,337.4,777.5,783.0,245.857143,39.727273,62.625,19.0,36.0,218.666667,0.0


In [66]:
piv = sale[['방송월', '방송시간', '판매량']].groupby(['방송월', '방송시간']).mean().reset_index()

In [67]:
pivT = piv.T
ema_s = pivT.ewm(span=4).mean()
ema_m = pivT.ewm(span=12).mean()
ema_l = pivT.ewm(span=26).mean()
macd = ema_s - ema_l
sig = macd.ewm(span=9).mean()

In [70]:
piv.ewm(span=26).mean()

,방송월,방송시간,판매량
0,1.000000,0.000000,316.145038
1,1.000000,0.519231,242.079345
2,1.000000,2.488418,225.063714
3,1.000000,3.749659,246.472780
4,1.000000,4.735331,282.891701
...,...,...,...
238,11.662461,13.322743,421.891157
239,11.687464,13.817355,410.760687
240,11.710615,14.349403,401.798947
241,11.732051,14.916113,384.662353


In [3]:
def engineering_timeSeries(df, dataset):
    
    df['방송날'] = df['방송일시'].dt.date
    df['방송날'] = pd.to_datetime(df['방송날'])
    
    if dataset == 'train':
        piv = pd.pivot_table(df, index = '상품군', columns = '방송날', values = '판매량', aggfunc=np.mean)
        pivT = piv.T

        ema_s = pivT.ewm(span=4).mean()
        ema_m = pivT.ewm(span=12).mean()
        ema_l = pivT.ewm(span=26).mean()
        macd = ema_s - ema_l
        sig = macd.ewm(span=9).mean()

        rol14 = pivT.fillna(0).rolling(14).mean()
        rol30 = pivT.fillna(0).rolling(30).mean()


        for tb, column in zip([ema_s, ema_m, ema_l, macd, sig, rol14, rol30], ['ema_s', 'ema_m', 'ema_l', 'macd', 'sig', 'rol14', 'rol30']):
            new_columns = list(map(lambda x : '_'.join((column, x)), tb.columns))
            tb.columns = new_columns

        timeS = pd.concat([ema_s, ema_m, ema_l, macd, sig, rol14, rol30], axis = 1)
        timeS = timeS.drop(timeS.columns[timeS.columns.str.contains('무형')], axis = 1)
        timeS = timeS.reset_index()
        timeS['방송날'] = pd.to_datetime(timeS['방송날'])
        
        joblib.dump(timeS,
                   os.path.join('..', '..', '0.Data', '01_제공데이터', 'data4time.pkl'))
        
    elif dataset == 'test':
        
        timeS = joblib.load(os.path.join('..', '..', '0.Data', '01_제공데이터', 'data4time.pkl'))
        df['방송날'] = df['방송날'].apply(lambda x : x - relativedelta(years=1))

    timeFE = ['ema_s', 'ema_m', 'ema_l', 'macd', 'sig', 'rol14', 'rol30']
    temp = pd.DataFrame(columns = timeFE)
    df = pd.concat([df, temp], axis = 1)
    
    for dt, cate in df[['방송날', '상품군']].drop_duplicates().values:
        try:
            df.loc[(df['방송날'] == dt) & (df['상품군'] == cate), ['ema_s', 'ema_m', 'ema_l', 'macd', 'sig', 'rol14', 'rol30']] = timeS.loc[timeS['방송날'] == dt, timeS.columns[timeS.columns.str.contains(cate)]].values
        except:
            continue
    df = df.drop('방송날', axis = 1)
    df[timeFE] = df[timeFE].astype(float)
    
    return df

In [95]:
piv = pd.pivot_table(sale, index = '상품군', columns = '방송일시', values = '판매량', aggfunc='sum')

In [79]:
pivT = piv.T
ema_s = pivT.ewm(span=4).mean()
ema_m = pivT.ewm(span=12).mean()
ema_l = pivT.ewm(span=26).mean()
macd = ema_s - ema_l
sig = macd.ewm(span=9).mean()

In [100]:
piv = pd.pivot_table(sale, index = '상품군', columns = '방송시간', values = '판매량', aggfunc=np.mean)

In [103]:
pivT = piv.T
ema_s = pivT.ewm(span=4).mean()
ema_m = pivT.ewm(span=12).mean()
ema_l = pivT.ewm(span=26).mean()
macd = ema_s - ema_l
sig = macd.ewm(span=9).mean()

In [115]:
sale

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,방송날,방송월,방송일,방송시간,판매량
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,1,1,6,53
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,1,1,6,110
2,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019-01-01,1,1,6,82
3,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019-01-01,1,1,6,175
4,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019-01-01,1,1,6,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,2020-01-01,1,1,0,0
38305,2020-01-01 00:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,2020-01-01,1,1,0,0
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,2020-01-01,1,1,1,0
38307,2020-01-01 01:00:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,2020-01-01,1,1,1,0


In [151]:
temp = pd.pivot_table(sale, index = '방송시간', columns = '상품군', values = '판매량', aggfunc = 'count').reset_index()
temp.columns = ['방송시간'] + list(map(lambda x : 'count_' + x, temp.columns[1:]))

In [153]:
list(temp.columns)

['방송시간',
 'count_가구',
 'count_가전',
 'count_건강기능',
 'count_농수축',
 'count_무형',
 'count_생활용품',
 'count_속옷',
 'count_의류',
 'count_이미용',
 'count_잡화',
 'count_주방',
 'count_침구']

In [162]:
temp = pd.pivot_table(sale, index = '방송날', columns = '상품군', values = '판매량', aggfunc = 'count').reset_index()

In [163]:
temp['방송날'] = pd.to_datetime(temp['방송날']).apply(lambda x : x + relativedelta(months=1))

In [164]:
temp.columns = ['방송날'] + list(map(lambda x : 'day_' + x, temp.columns[1:]))

In [165]:
list(temp.columns)

['방송날',
 'day_가구',
 'day_가전',
 'day_건강기능',
 'day_농수축',
 'day_무형',
 'day_생활용품',
 'day_속옷',
 'day_의류',
 'day_이미용',
 'day_잡화',
 'day_주방',
 'day_침구']

In [166]:
temp

,방송날,day_가구,day_가전,day_건강기능,day_농수축,day_무형,day_생활용품,day_속옷,day_의류,day_이미용,day_잡화,day_주방,day_침구
0,2019-02-01,NaN,6.0,NaN,9.0,NaN,NaN,13.0,24.0,3.0,NaN,3.0,NaN
1,2019-02-02,NaN,24.0,3.0,18.0,NaN,3.0,20.0,11.0,NaN,NaN,27.0,NaN
2,2019-02-03,NaN,18.0,3.0,12.0,NaN,3.0,17.0,13.0,6.0,9.0,15.0,NaN
3,2019-02-04,NaN,6.0,3.0,12.0,NaN,6.0,11.0,11.0,3.0,NaN,30.0,NaN
4,2019-02-05,NaN,24.0,NaN,8.0,NaN,NaN,16.0,21.0,6.0,6.0,18.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2020-01-28,NaN,NaN,3.0,11.0,12.0,18.0,3.0,12.0,3.0,23.0,18.0,NaN
362,2020-01-29,NaN,3.0,NaN,6.0,11.0,11.0,9.0,3.0,9.0,9.0,21.0,NaN
363,2020-01-30,NaN,3.0,6.0,15.0,1.0,6.0,9.0,12.0,6.0,6.0,6.0,NaN
364,2020-01-31,9.0,6.0,3.0,6.0,4.0,21.0,6.0,9.0,NaN,6.0,32.0,NaN


In [158]:
temp = pd.pivot_table(sale, index = '방송일시', columns = '상품군', values = '판매량', aggfunc = 'count').reset_index()

In [159]:
temp.columns = ['방송일시'] + list(map(lambda x : 'hour_' + x, temp.columns[1:]))

In [161]:
list(temp.columns)

['방송일시',
 'hour_가구',
 'hour_가전',
 'hour_건강기능',
 'hour_농수축',
 'hour_무형',
 'hour_생활용품',
 'hour_속옷',
 'hour_의류',
 'hour_이미용',
 'hour_잡화',
 'hour_주방',
 'hour_침구']